In [4]:
from keras.applications import  inception_v3
from keras import backend as K

K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights='imagenet',include_top=False)
#Incepton的网络不含有全连接层，imagenet为Imagenet的网络权重
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_94 (BatchNo (None, None, None, 3 96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, None, None, 3 0           batch_normalization_94[0][0]     
_______________________________________________________________________________________

In [25]:
layer_contributions = {
    'mixed2':0.2,
    'mixed3':3.,
    'mixed4':2.,
    'mixed5':1.5
}

In [26]:
layer_dict = dict([(layer.name,layer) for layer in model.layers])

loss =  K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output
    
    scaling = K.prod(K.cast(K.shape(activation),'float32'))
    loss = loss + coeff * K.sum(K.square(activation[:,2:-2,2:-2, :])) / scaling#不支持

In [29]:
import tensorflow as tf


dream = model.input

with tf.GradientTape() as gtape:
    grads = gtape.gradient(loss, dream)
    
print(grads)
grads /= K.maximum(K.mean(K.abs(grads)),1e-7)  #梯度标准化

outputs = [loss,grads]
fetch_loss_and_grads = K.funtion([dream],outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_value = outs[0]
    return loss_value,grad_value

#梯度上升函数的实现，此函数为运行iteration次梯度上升
def gradient_ascent(x,iterations,step,max_loss=None):
    for i in range(iterations):
        loss_value,grad_value = eval_loss_and_grads(x)
        if max_loss is not None and loss_value>max_loss:
            break
        print('...Loss value at',i,':',loss_value)
        x += step * grad_values
    return x

None


ValueError: Attempt to convert a value (None) with an unsupported type (<class 'NoneType'>) to a Tensor.